In [2]:
import hashlib
import logging
import os
import re

import bs4
import numpy as np
import torch
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader, WebBaseLoader
from langchain_core.documents import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import (CharacterTextSplitter,
                                      RecursiveCharacterTextSplitter)
from pinecone import Pinecone, ServerlessSpec
from pinecone.data.index import Index
from tqdm import tqdm
from typing_extensions import List, TypedDict

In [3]:
# from rag_pipeline import RetrieverModel
DEVICE = ('cuda' if torch.cuda.is_available() else
          'mps' if torch.backends.mps.is_available() else 'cpu')
DEVICE

'mps'

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='all-mpnet-base-v2',
                                   model_kwargs={'device': DEVICE})
# dimension = len(embeddings.embed_documents(['test'])[0])
# dimension

768

In [ ]:
pc = Pinecone()
pc_index = pc.Index('chunking-test')
vector_store = PineconeVectorStore(pc_index, embeddings)

In [5]:
loader = DirectoryLoader('raw_data', glob='**/*.txt',
                         show_progress=True, use_multithreading=False)
docs = loader.load()

  0%|          | 0/136 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 136/136 [01:08<00:00,  1.98it/s]


In [7]:
text_splitter = SemanticChunker(embeddings)
chunks = [chunk for doc in tqdm(docs)
          for chunk in text_splitter.split_documents([doc])]

  0%|          | 0/136 [00:00<?, ?it/s]

  3%|▎         | 4/136 [14:25<7:55:50, 216.29s/it] 


KeyboardInterrupt: 

In [ ]:
vector_store.add_documents(chunks)